# preprocessCoreLogic #
**Author:** Andrew Larkin <br>
Developed for the HEI Transit Study <br>
**Summary:** given a large core logic csv file, reduce the variables to those needed for the wind analysis and restrict to within 5km of maternal residences

## part 1: import libraries and define global constants ##

In [1]:
import pandas as ps
import os
import arcpy
import const as gConst
arcpy.env.overwriteOutput=True

In [2]:
PARENT_FOLDER = const.WIND_FOLDER + "Corelogic_Building_Years/"
CORE_LOGIC_FILE = PARENT_FOLDER + "CoreLogicData/TaxCompiled.csv"

## part 2: load corelogic into python and remove unneeded variables and records without GIS coordinates ###

In [20]:
coreLogicData = ps.read_csv(CORE_LOGIC_FILE)
coreLogicData.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,7,11,15,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,fipscode,apnparcelnumberunformatted,apnsequencenumber,compositepropertylinkagekey,previousparcelnumber,situshousenumber,situshousenumbersuffix,situshousenumber2,situsdirection,situsstreetname,...,apnparcelnumberunformatted_03,previousparcelnumber_03,apnparcelnumberunformatted_02,previousparcelnumber_02,apnparcelnumberunformatted_01,previousparcelnumber_01,lastobserved,firstobserved,missingyearbuilt,disappears
0,48001,M100045,1,48001M100045 ...,NaN,1838,NaN,NaN,E,FM 323,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,0,1
1,48001,M100170,1,48001M100170 ...,NaN,620,NaN,NaN,NaN,AN COUNTY ROAD 2419,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,0,1
2,48001,M100343,1,48001M100343 ...,NaN,1769,NaN,NaN,NaN,AN COUNTY ROAD 2419,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,0,1
3,48001,M100353,1,48001M100353 ...,NaN,4671,NaN,NaN,NaN,FM 322,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,0,1
4,48001,M100408,1,48001M100408 ...,NaN,2463,NaN,NaN,NaN,AN COUNTY ROAD 318,...,NaN,NaN,NaN,NaN,NaN,NaN,12,12,0,1


In [21]:
print(coreLogicData.keys())
print(coreLogicData.count()[0])

Index(['fipscode', 'apnparcelnumberunformatted', 'apnsequencenumber',
       'compositepropertylinkagekey', 'previousparcelnumber',
       'situshousenumber', 'situshousenumbersuffix', 'situshousenumber2',
       'situsdirection', 'situsstreetname', 'situsmode', 'situsquadrant',
       'situsunitnumber', 'situscity', 'situsstate', 'situszipcode',
       'yearbuilt', 'storiesnumber', 'blocklevellatitude',
       'blocklevellongitude', 'parcellevellatitude', 'parcellevellongitude',
       'apnparcelnumberunformatted_12', 'previousparcelnumber_12',
       'apnparcelnumberunformatted_11', 'previousparcelnumber_11',
       'apnparcelnumberunformatted_10', 'previousparcelnumber_10',
       'apnparcelnumberunformatted_09', 'previousparcelnumber_09',
       'apnparcelnumberunformatted_08', 'previousparcelnumber_08',
       'apnparcelnumberunformatted_07', 'previousparcelnumber_07',
       'apnparcelnumberunformatted_06', 'previousparcelnumber_06',
       'apnparcelnumberunformatted_05', 'previ

In [22]:
coreLogicData['yearbuilt'] = coreLogicData['yearbuilt'].fillna(-1)
coreLogicData['yearbuilt'] = coreLogicData['yearbuilt'].astype(int)

In [23]:
reduced = coreLogicData[['yearbuilt','storiesnumber','parcellevellatitude','parcellevellongitude']]
reduced = reduced[reduced['parcellevellatitude']>0]
reduced = reduced[reduced['parcellevellongitude']>-200]
reduced['storiesnumber'] = reduced['storiesnumber'].fillna(-1)
reduced['storiesnumber'] = reduced['storiesnumber'].astype(int)
reduced.to_csv(PARENT_FOLDER + "CoreLogicData/reduced.csv",index=False)

## part 3: load reduced core logic records into GIS  ##

In [5]:
#  These values based on your script
XFieldName = 'parcellevellongitude'
YFieldName = 'parcellevellatitude'
outFolder = PARENT_FOLDER + "/"
spatialRef = arcpy.SpatialReference(4326)
csvFilePath = PARENT_FOLDER + "CoreLogicData/reduced.csv"
coreLogicLayer = "coreLogicLayer"
coreLogicShapefile = const.WIND_FOLDER + "temp/intermediateProducts.gdb"
arcpy.MakeXYEventLayer_management(csvFilePath, XFieldName, YFieldName, coreLogicLayer, spatial_reference=spatialRef)
arcpy.FeatureClassToShapefile_conversion(coreLogicLayer, coreLogicShapefile)

<Result 'X:\\BPHS\\HEI_Transit_Study\\Andy\\intermediateProducts'>

## part 4: restrict core logic data points to within 5km of maternal residence locations ##

In [ ]:
residenceLocations = const.WIND_FOLDER + "Birth_Addresses_Wind/births_shapefile/Births0716_Wind.shp"
outTable = const.WIND_FOLDER + "temp/intermediateProducts.gdb/coreNearResidence"
arcpy.GenerateNearTable_analysis(coreLogicLayer, residenceLocations, outTable, "5000 Meters")